In [3]:
import pickle

import pandas as pd
from tqdm.auto import tqdm

In [4]:
tqdm.pandas()

In [5]:
df = pd.read_parquet('data/df_pico_mapped.pqt', engine='fastparquet')

In [6]:
train = pd.read_parquet('data/train_split.pqt')

In [7]:
from age_groups import age_to_group

train['age'] = train['age'].apply(age_to_group)

In [8]:
idx_to_age = train['age'].to_dict()
idx_to_male = train['is_male'].to_dict()

In [9]:
df = df[df['user_id'].isin(train.index)]

In [10]:
from collections import defaultdict

occurs = defaultdict(lambda: {'age': {i: 0 for i in range(7)}, 'is_male': {True: 0, False: 0}})
occurs

defaultdict(<function __main__.<lambda>()>, {})

In [11]:
for row in tqdm(df.itertuples()):
    occurs[row.url_host]['age'][idx_to_age[row.user_id]] += row.request_cnt
    occurs[row.url_host]['is_male'][idx_to_male[row.user_id]] += row.request_cnt

0it [00:00, ?it/s]

In [12]:
occurs = dict(occurs)

In [13]:
with open('data/statistics.pkl', 'wb') as f:
    f.write(pickle.dumps(occurs))